In [54]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile 
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range 


In [55]:
pickle_file='notMNIST.pickle'


with open(pickle_file , 'rb') as f:
    save=pickle.load(f)
    train_dataset=save ['train_dataset']
    train_labels=save ['train_labels']
    valid_dataset=save ['valid_dataset']
    valid_labels=save ['valid_labels']
    test_dataset=save ['test_dataset']
    test_labels=save ['test_labels']
    del save #to help compiler to free up space 
    print('train_dataset', train_dataset.shape , train_labels.shape)
    print('validation set ', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape, test_labels.shape)

train_dataset (200000, 28, 28) (200000,)
validation set  (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [56]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [57]:
log_dir='Hidden_1layer_log'
def accuracy(predictions, labels):
    return(100.0 * np.sum(np.argmax(predictions, 1)== np.argmax(labels, 1))/ predictions.shape[0])
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
    tf.gfile.MakeDirs(log_dir)

In [58]:
batch_size=128
num_nodes=1024
#SGDSTEP=0.5
#NumberofLayers=2 #(front layer+1 hidden layer)

'''
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial:tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
        '''
    
graph=tf.Graph()
with graph.as_default():
    # Input data.For the training data,we use a placeholder to fed with a training mini batch
    #at runtime with a training min i batch
    tf_train_dataset= tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    with tf.name_scope('weights1'):
        weights_1=tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes]))
        variable_summaries(weights_1)
    with tf.name_scope('biases1'):
        biases_1=tf.Variable(tf.zeros([num_nodes]))
        variable_summaries(biases_1)
    with tf.name_scope('weights2'):
        weights_2=tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
        variable_summaries(weights_2)
    with tf.name_scope('biases2'):
        biases_2=tf.Variable(tf.zeros(num_labels))
        variable_summaries(biases_2)
    with tf.name_scope('W1x_plus_b1'):
        preactivate = tf.matmul(tf_train_dataset,weights_1)+ biases_1
        tf.summary.histogram('pre_activations1', preactivate)
    relu_layer=tf.nn.relu(preactivate)
    tf.summary.histogram('activations', relu_layer)
    
    #training Computation.
    
    logits=tf.matmul(relu_layer,weights_2)+biases_2
    #loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    
    with tf.name_scope('cross_entropy'):
        # The raw formulation of cross-entropy,
        #
        # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
        #                               reduction_indices=[1]))
        #
        # can be numerically unstable.
        #
        # So here we use tf.nn.softmax_cross_entropy_with_logits on the
        # raw outputs of the nn_layer above, and then average across
        # the batch.
        with tf.name_scope('total'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    tf.summary.scalar('cross_entropy', loss)

    
    #optimizer
    with tf.name_scope('train'):
        optimizer=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predictions for training
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1)+biases_1),weights_2)+biases_2)
    test_predicton= tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1),weights_2)+biases_2)
    
    
   
    
   
    print(graph)

In [62]:
num_steps=30001


 

with tf.Session(graph=graph) as session:
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir + '/train',session.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    tf.global_variables_initializer().run()
    print("Initialized")
    c=0
    step_array=[]
    accuracy_array=[]
    
    for step in range(num_steps):
        #Pick an offset within taining data. which has been randomized.
        # Note: we could use better randomization across epochs
        offset= (step*batch_size) % (train_labels.shape[0] - batch_size) # we have used train_labels.shape[0] it is equal to the toatal no of image in the datasets
        
        #generate a minibatch , as SGD requires them
        batch_data= train_dataset[offset:(offset+batch_size),:]
        batch_labels= train_labels[offset:(offset+batch_size),:]
        
        #Prepare a dictionary telling the session where to feed the minibatch
        # The Key of the dictinary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict={tf_train_dataset: batch_data,tf_train_labels: batch_labels}
        _,summary, l,predictions = session.run( [optimizer,merged,loss, train_prediction], feed_dict=feed_dict)
        test_writer.add_summary(summary, step)
        '''
        if (step % 500==0):
            miniAccuracy=accuracy( valid_prediction.eval(), valid_labels)
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accurracy: %.1f%%" % miniAccuracy)
            print (c)
            step_array.append(step)
            accuracy_array.append(miniAccuracy)
            c=c+1
            plt.plot(step,miniAccuracy)
        print("Test Accuracy: %.1f%%" % accuracy(test_predicton.eval(),test_labels))
        '''
        print(accuracy(test_predicton.eval(),test_labels))
        
session.close()

Initialized
29.09
36.1
43.06
52.57
52.81
73.68
78.07
77.56
78.86
77.63
82.33
80.49
80.47
83.0
79.79
78.22
80.66
77.8
80.09
78.0
80.83
81.61
73.43
81.67
83.4
85.44
85.72
78.94
82.51
82.18
83.15
83.57
82.22
81.3
76.75
83.7
84.24
84.07
84.6
84.64
84.55
84.88
82.36
85.07
82.85
80.62
85.32
84.3
82.87
82.35
84.71
83.7
85.31
85.81
85.31
85.73
80.98
81.03
85.14
82.75
76.91
83.09
85.37
82.84
85.42
82.53
81.27
83.74
84.72
83.42
84.72
75.56
81.64
83.59
81.95
84.36
84.97
85.49
85.27
84.62
83.63
81.59
83.49
84.79
84.63
85.31
83.37
83.24
85.82
84.93
85.6
82.28
84.9
84.16
85.2
85.03
83.12
84.74
85.95
84.44
84.67
85.25
86.15
85.81
86.79
86.51
85.22
84.88
85.04
84.05
84.98
85.81
86.08
82.66
83.59
85.03
83.67
85.62
85.73
81.11
79.61
83.13
86.06
83.61
84.9
85.83
86.78
86.47
84.7
83.64
86.01
86.29
86.87
86.54
87.17
86.99
82.46
83.53
85.72
85.4
84.91
86.34
84.39
84.01
83.35
80.53
83.87
85.0
84.32
86.92
87.65
83.83
83.67
85.98
84.89
86.42
85.72
86.58
84.33
84.51
85.82
83.83
86.29
86.68
87.01
85.76
85.74
86.

KeyboardInterrupt: 